In [1]:
from pydantic import BaseModel, Field
from typing import List
import guidance
import os


In [24]:
# # vllm hosted model using LiteLLM

# litellm_desc = {
#     "model_name": "Qwen/Qwen3-30B-A3B-Instruct-2507",
#     "litellm_params": {  # params for litellm completion/embedding call
#         "model": "hosted_vllm/Qwen/Qwen3-30B-A3B-Instruct-2507",
#         "api_key": os.environ.get("VLLM_API_KEY", "NO_KEY"), # set your vLLM API key if needed
#         "api_base": "http://localhost:8000/v1", # change to your vLLM API base URL
#     },
# }
# base_lm = guidance.models.experimental.LiteLLM(model_description=litellm_desc, echo=True)

In [2]:
base_lm = guidance.models.experimental.SglangModel(model="Qwen/Qwen3-30B-A3B-Instruct-2507", base_url="http://127.0.0.1:30000/v1" ,echo=True, api_key = os.environ.get("OPENAI_API_KEY", "NO_KEY"))

In [3]:
# def run_gen_test(lm):
#     with guidance.user():
#         lm += "What is the capital of France? and its population?"
#         lm += "Format your answer as follows: Capital: <capital>, Population: <population>"

#     with guidance.assistant():
#         lm += guidance.gen(max_tokens=1024, temperature=0.7, name="answer")
#         print(lm["answer"])

# run_gen_test(base_lm)

In [4]:
from pydantic import BaseModel, Field
from typing import List
import guidance
from guidance import models, system, user, assistant, json as gen_json, select
import json
from enum import Enum


class FirstPrinciples(BaseModel):
    absolute_truths: List[str] = Field(
        ...,
        description="Look at the problem statement and list the things that you know that are absolute truths."
    )


class GroundUpReasoning(BaseModel):
    """Model for the LLM-generated reasoning plan."""
    reason: str = Field(
        ...,
        description="Based on the known truths reason in a concise sharp manner from ground up to address the problem."
    )

class Reflection(BaseModel):
    """Model for an individual reasoning step execution."""
    evaluation: str = Field(
        ...,
        description="Reflect on your reasoning and identify gaps / uncertainties / mistakes."
    )

class ReasoningConclusion(BaseModel):
    """Model for the final reasoning conclusion."""
    conclusion: str = Field(
        ...,
        description="Address the identified gaps in the previous step if any and deduce the final conclusion."
    )

def dynamic_final_answer_schema(Options):
    class FinalAnswer(BaseModel):
        answer: Options = Field(..., description="Final answer.")

    return FinalAnswer


@guidance()
def _first_principles(llm):
    with assistant():
        llm += gen_json(
            name="first_principles",
            schema=FirstPrinciples,
            max_tokens=5000
        )
    return llm

@guidance()
def _generate_groundup_reasoning(llm):
    """
    Generate a custom reasoning plan tailored to the query.
    """
    with assistant():
        llm += gen_json(
            name="groundup_reasoning",
            schema=GroundUpReasoning,
            max_tokens=5000  # Reduced to encourage conciseness
        )
    return llm

@guidance()
def _reflection(llm):
    with assistant():
        llm += gen_json(
            name="reflection",
            schema=Reflection,
            max_tokens=5000
        )
    return llm
    

@guidance()
def _reasoning_conclusion(llm):
    with user():
        llm += """Based on the information, make logical deductions."""
    
    with assistant():
        llm += gen_json(
            name="reasoning_conclusion",
            schema=ReasoningConclusion,
            max_tokens=5000
        )

    return llm

@guidance
def dynamic_structured_reasoning(llm, query, output_choices):
    Options = Enum("Choices", {choice: choice for choice in output_choices})
    final_answer_schema = dynamic_final_answer_schema(Options)

    with system():
        llm += """You are an expert in problem solving. For any query, reason from first principles to solve the query. 
        Structure all outputs as JSON. Be very brief in your steps, tending towards concise but complete responses.
        """

    with user():
        llm += f"""Query: {query}
        Look at the problem statement and list the things that you know that are absolute truths -
        """

    llm += _first_principles()

    with user():
        llm += f"""
        Based on the known truths reason from ground up to address the problem, be extremely brief and to the point in your reasoning -
        """

    llm += _generate_groundup_reasoning()

    with user():
        llm += f"""
        Reflect to identify gaps / uncertainties / mistakes in reasoning process -
        """

    llm += _reflection()

    with user():
        llm += f"""
        Make logical deductions based on the information you have to arrive at a conclusion -
        """

    llm += _reasoning_conclusion()

    with user():
        llm += f"""
        Your final answer -
        """

    with assistant():
        llm += gen_json(
            name="answer",
            schema=final_answer_schema,
            max_tokens=1000)

    return llm

In [5]:
def generate(query, output_choices, lm = None):
    lm = lm or base_lm
    result_lm = lm + dynamic_structured_reasoning(
                    query=query,
                    output_choices = output_choices
                )

    answer = json.loads(result_lm.get("answer"))['answer']
    
    groundup_reasoning = json.loads(result_lm.get("groundup_reasoning"))['reason']
    first_principles = json.loads(result_lm.get("first_principles"))['absolute_truths']
    reflection = json.loads(result_lm.get("reflection"))['evaluation']   
    reasoning_conclusion = json.loads(result_lm.get("reasoning_conclusion"))['conclusion']
    return first_principles, groundup_reasoning, reflection, reasoning_conclusion, answer


In [6]:
from datasets import load_dataset

# Load the BBH dataset from Hugging Face
dataset = load_dataset("WildEval/ZebraLogic", 'mc_mode')



In [7]:
from json import JSONDecodeError


def evaluate_reasoning(row):
    """
    Placeholder function to evaluate your reasoning system on a single row.
    Replace with your dynamic_structured_reasoning function.
    """
    
    # from guidance.models.experimental import SglangModel
    # lm = SglangModel(
    #     model="Qwen/Qwen3-30B-A3B-Instruct-2507",
    #     base_url="http://127.0.0.1:30000/v1",
    #     echo=True,
    #     api_key=os.environ.get("OPENAI_API_KEY", "NO_KEY"),
    # )
    puzzle = row['puzzle']
    question = row['question']
    query = puzzle + '\n\n' + question
    choices = row['choices']
    target = row['answer']
    print(row['id'])
    # Your reasoning system should return a predicted answer
    try:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = generate(query, choices)
    except JSONDecodeError as e:
        first_principles, groundup_reasoning, reflection, reasoning_conclusion, pred = ["JSONDecodeError"]*5
    return {
        'correct': pred == target,
        'first_principles': first_principles,
        'groundup_reasoning': groundup_reasoning,
        'reflection': reflection,
        'reasoning_conclusion': reasoning_conclusion,
        'predicted_answer': pred, 
    }


In [8]:
# from concurrent.futures import ThreadPoolExecutor, as_completed
# import pandas as pd

# # df is already df.iloc[600:650]  (keeps original index)
# def run_parallel_preserve_index(df, workers=16):
#     idx_to_row = df.to_dict(orient="index")  # {index: rowdict}
#     results = {}

#     with ThreadPoolExecutor(max_workers=workers) as ex:
#         futs = {ex.submit(evaluate_reasoning, row): idx
#                 for idx, row in idx_to_row.items()}
#         for fut in as_completed(futs):
#             idx = futs[fut]
#             results[idx] = fut.result()

#     res_df = pd.DataFrame.from_dict(results, orient="index")
#     # ensure order matches df (and keep original index)
#     res_df = res_df.reindex(df.index)
#     return df.join(res_df)

# df = pd.DataFrame(dataset['test'])
# # df.head()
# # CURRENTLY FINISHED - [:600]

# df = df.iloc[600:605]
# df = run_parallel_preserve_index(df, workers=16)


In [9]:
# import pandas as pd
# df = pd.DataFrame(dataset['test'])
# len(df)

In [16]:
import pandas as pd

df = pd.DataFrame(dataset['test'])
# df.head()
# CURRENTLY FINISHED - [:1300]

df = df.iloc[1300:1600]

res = df.apply(lambda r: pd.Series(evaluate_reasoning(r)), axis=1)
df = pd.concat([df, res], axis=1)

lgp-test-4x6-14#mc-16


StitchWidget(initial_height='auto', initial_width='100%', srcdoc='<!doctype html>\n<html lang="en">\n<head>\n …

lgp-test-6x4-26#mc-5
lgp-test-2x3-1#mc-5
lgp-test-4x5-37#mc-0
lgp-test-2x3-25#mc-3
lgp-test-3x6-17#mc-11
lgp-test-4x5-21#mc-1
lgp-test-5x6-14#mc-25
lgp-test-5x5-1#mc-18
lgp-test-4x4-19#mc-6
lgp-test-2x5-18#mc-8
lgp-test-3x3-29#mc-1
lgp-test-2x5-18#mc-6
lgp-test-5x6-25#mc-8
lgp-test-3x5-34#mc-5
lgp-test-4x3-11#mc-3
lgp-test-6x5-24#mc-12
lgp-test-2x6-22#mc-10
lgp-test-3x3-16#mc-6
lgp-test-4x6-6#mc-5
lgp-test-5x4-24#mc-1
lgp-test-2x6-17#mc-2
lgp-test-3x6-29#mc-1
lgp-test-6x5-28#mc-7
lgp-test-6x4-30#mc-7
lgp-test-4x5-26#mc-12
lgp-test-2x5-32#mc-2
lgp-test-5x6-25#mc-27
lgp-test-4x5-20#mc-7
lgp-test-6x3-11#mc-10
lgp-test-5x3-37#mc-8
lgp-test-3x6-34#mc-16
lgp-test-6x3-26#mc-7
lgp-test-5x6-27#mc-13
lgp-test-6x5-11#mc-0
lgp-test-5x3-34#mc-3
lgp-test-2x6-5#mc-5
lgp-test-4x6-38#mc-19
lgp-test-2x3-11#mc-5
lgp-test-3x4-31#mc-4
lgp-test-6x6-24#mc-9
lgp-test-4x4-12#mc-11
lgp-test-4x6-7#mc-15
lgp-test-5x4-30#mc-12
lgp-test-6x5-16#mc-3
lgp-test-2x4-14#mc-6
lgp-test-4x5-33#mc-0
lgp-test-6x4-20#mc-8
lgp-

In [17]:
df

,id,puzzle,question,choices,answer,created_at,correct,first_principles,groundup_reasoning,reflection,reasoning_conclusion,predicted_answer
1300,lgp-test-4x6-14#mc-16,"There are 4 houses, numbered 1 to 4 from left ...",What is Hobby of the person who lives in House 3?,"[painting, gardening, photography, cooking]",painting,2024-07-03T21:21:31.333516,True,"[Arnold drinks tea (from clue 7)., The tea dri...","Peter is in the colonial house (clue 15), whic...",The reasoning is logically consistent and all ...,The hobby of the person in House 3 is painting.,painting
1301,lgp-test-6x4-26#mc-5,"There are 6 houses, numbered 1 to 6 from left ...",What is Pet of the person who lives in House 2?,"[dog, bird, cat, hamster, fish, rabbit]",rabbit,2024-07-03T21:21:31.373484,True,"[House 3: Carol, House 2: Birthday = May, Hous...","House 2: Peter, Colonial, Birthday = May. Pet:...",Initial reasoning correctly identified key con...,House 2 has the pet Rabbit.,rabbit
1302,lgp-test-2x3-1#mc-5,"There are 2 houses, numbered 1 to 2 from left ...",What is Animal of the person who lives in Hous...,"[cat, horse]",cat,2024-07-03T21:21:31.299297,True,"[House 1 is to the left of House 2., Each hous...",House 1 has the horse (clue 2). So House 2 has...,The reasoning correctly uses clue 2: horse is ...,The animal in House 2 is the cat.,cat
1303,lgp-test-4x5-37#mc-0,"There are 4 houses, numbered 1 to 4 from left ...",What is Name of the person who lives in House 1?,"[Eric, Peter, Arnold, Alice]",Arnold,2024-07-03T21:21:31.330325,True,"[House 3 is occupied by Alice (from clue 8)., ...",House 3: Alice (clue 8). House 1: Desert smoot...,All steps are logically consistent and support...,The person in House 1 is Arnold.,Arnold
1304,lgp-test-2x3-25#mc-3,"There are 2 houses, numbered 1 to 2 from left ...",What is Name of the person who lives in House 2?,"[Arnold, Eric]",Arnold,2024-07-03T21:21:31.300004,True,"[There are 2 houses, numbered 1 (left) and 2 (...",House 1 is left of House 2. Clue 2: Eric is di...,Reasoning is logically sound: Clue 2 forces Er...,The person in House 2 is Arnold.,Arnold
...,...,...,...,...,...,...,...,...,...,...,...,...
1595,lgp-test-6x2-19#mc-1,"There are 6 houses, numbered 1 to 6 from left ...",What is Food of the person who lives in House 1?,"[stir fry, spaghetti, grilled cheese, pizza, s...",stew,2024-07-03T21:21:31.363714,True,"[House 1: Bob, House 3: Alice, House 4: Arnold...",Bob is in House 1 (Clue 5). Carol loves stir f...,Initial reasoning assumed stir fry cannot be i...,"House 1: Bob, Food: stew",stew
1596,lgp-test-3x4-32#mc-2,"There are 3 houses, numbered 1 to 3 from left ...",What is PhoneModel of the person who lives in ...,"[samsung galaxy s21, iphone 13, google pixel 6]",google pixel 6,2024-07-03T21:21:31.311068,True,"[Peter uses an iPhone 13 (from clue 1)., Peter...",Peter uses iPhone 13 and has Sept birthday. Er...,Initial reasoning correctly identified constra...,The person in House 1 uses the Google Pixel 6.,google pixel 6
1597,lgp-test-5x4-39#mc-7,"There are 5 houses, numbered 1 to 5 from left ...",What is Food of the person who lives in House 2?,"[spaghetti, grilled cheese, stew, stir fry, pi...",stew,2024-07-03T21:21:31.347796,False,[The person who is very short is in the fifth ...,House 3: tall (Clue 13). House 3: grilled chee...,Initial error: Assumed Bob must be in 4 after ...,House 2's food is cooking.,grilled cheese
1598,lgp-test-5x4-16#mc-10,"There are 5 houses, numbered 1 to 5 from left ...",What is Cigar of the person who lives in House 3?,"[pall mall, blue master, dunhill, blends, prince]",blends,2024-07-03T21:21:31.345104,True,"[House 4 has the Pall Mall smoker (Clue 10)., ...","Bob ∈ {2,3,4}, Blue Master smoker is left of B...",The reasoning process is logically consistent ...,The cigar in House 3 is blends.,blends


In [18]:
df['correct'].value_counts()

correct
True     212
False     88
Name: count, dtype: int64

In [19]:
path = './zebralogic_eval.csv'
df.to_csv(path, mode="a", index=False, header=not os.path.exists(path))

In [22]:
dub = pd.read_csv(path)


In [23]:
dub['correct'].value_counts()

correct
True     1173
False     427
Name: count, dtype: int64